In [28]:
#Speech to text with google and ffmpeg wav2flac

import speech_recognition as sr
import os
from google.cloud import speech
#from google.cloud.speech import enums
#from google.cloud import speech_V1
#from google.cloud.speech_V1 import enums
#from google.cloud.speech_V1 import types
#from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
#from google_auth_oauthlib.flow import InstalledAppFlow
#from googleapiclient.discovery import build
#from googleapiclient.errors import HttpError


# wav to flac converter
def convert_to_flac(source_file_name):
    import subprocess
    subprocess.call(['ffmpeg', '-i', source_file_name, '-c:a', 'flac', '-y', '-ar', '16000', '-f', 'flac', 'audio.flac'])
#    subprocess.call(['avconv', '-i', source_file_name, '-y', '-ar', '48000', '-ac', '1', 'audio.flac'])
    return os.path.abspath('audio.flac')


# upload to GCS
def upload_file_to_gcs(bucket_name, source_file_name, destination_blob_name):
    from google.cloud import storage
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f'File {source_file_name} uploaded to gs://{bucket_name}/{destination_blob_name}')


# audio to text
def transcribe_gcs(gcs_uri):
#    client = speech_v1p1beta1.SpeechClient()
    client = speech.SpeechClient()
#    audio = types.RecognitionAudio(uri=gcs_uri)
#    config = types.RecognitionConfig(
    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
#        encoding=enums.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
        encoding=speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,

        sample_rate_hertz=16000,
        language_code='en-US')
#    response = client.long_running_recognize(config, audio)
    response = client.long_running_recognize(config=config, audio=audio)
    print('Waiting for operation to complete...')
    response = response.result(timeout=90)
    transcript = ''
    for result in response.results:
        transcript += result.alternatives[0].transcript

    return transcript

# record audio from mic
def record_audio():
    r = sr.Recognizer()
    print("Speak now...")
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, 0.5) # noise reducing
        audio = r.listen(source)
    with open("output.wav", "wb") as f:
        f.write(audio.get_wav_data())
    print("Audio recording saved as output.wav")
    return os.path.abspath('output.wav')

# main
def main():
    bucket_name = 'ivandaf_speech_bucket'
    audio_file = record_audio()
    flac_file = convert_to_flac(audio_file)
    gcs_file_name = os.path.basename(flac_file)
    destination_blob_name = gcs_file_name
    upload_file_to_gcs(bucket_name, flac_file, destination_blob_name)
    gcs_uri = f'gs://{bucket_name}/{destination_blob_name}'
    transcription = transcribe_gcs(gcs_uri)
    print("\nTranscript: ", transcription)

# execute
if __name__ == "__main__":
    main()


Speak now...
Audio recording saved as output.wav


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

File /Users/ivandaf/Downloads/audio.flac uploaded to gs://ivandaf_speech_bucket/audio.flac
Waiting for operation to complete...

Transcript:  hello world
